In [1]:
import requests
import re
import time
import string
import random
import math
import pandas as pd
from urllib.parse import urlparse
from collections import Counter
from bs4 import BeautifulSoup
from pymystem3 import Mystem

In [2]:
def crawler(urls):
    links_tree = set()
    for url in urls:
        if len(texts) >= 100:
            return
    
        if url in visited:
            continue
        visited.add(url)
            
        try:
            #получаем контент страницы и декодируем его
            html = requests.get(url).content.decode()
        except:
            time.sleep(1)
            continue
        
        #парсинг страницы
        soup = BeautifulSoup(html)
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text(' ').strip().replace('\xa0', ' ').replace('\n', '').replace('\r', '').replace('\t', '')
        text = re.sub(' +', ' ', text)
    
        if len(text.split(' ')) >= 10:
            texts[url] = text
            
        parsed = urlparse(url)
        base = f"{parsed.scheme}://{parsed.netloc}"    
        links = re.findall('''<a\s+(?:[^>]*?\s+)?href="([^"]*)"''', html) 
        
        for i, link in enumerate(links):    
            if not urlparse(link).netloc:    
                link_with_base = base + link    
                links[i] = link_with_base
                
        links_tree.update(set(filter(lambda x: 'mailto' not in x, links)))
    
    crawler(links_tree)

In [3]:
texts = {}
visited = set()
start = 'https://m.habr.com/ru/all/'

crawler([start])

In [4]:
index = {i:k for i, k in enumerate(texts)} 

In [5]:
with open('index.txt', 'w') as f:
    for i in index:
        f.write(f"{i}: {index[i]}\n")

In [6]:
from openpyxl import load_workbook
import pandas as pd
import os, pymorphy2

In [7]:
words_all = {}
lemma_words = {}
lemma_normal_forms = {}

In [8]:
def get_sparsed_list(word_list):
    # получить список всех словоформ
    morph, lst = pymorphy2.MorphAnalyzer(), []
    for word in word_list:
        lemma = ""
        for x in morph.parse(word)[0].lexeme:
            lemma+=x.word + " "
        lst.append(f"{word} {lemma}\n")
    return lst

In [9]:
texts_lemmatized = {}

for url in texts:
    text = texts[url]
    # оставляем только слова
    text = re.sub(r'[^a-zA-Zа-яА-Я]+', ' ', text)
   
    words = text.split(' ')
    words_all[url]=words
    
    texts_lemmatized[url]=get_sparsed_list(words_all[url])  

In [10]:
with open('words.txt', 'w') as f:
    for url in words_all:
        for i, score in enumerate(words_all[url]):
            f.write(f"{score}\n")

In [11]:
with open('lemma_words.txt', 'w') as f:
    for url in texts_lemmatized:
        for i, score in enumerate(texts_lemmatized[url]):
            f.write(f"{score} {url}\n")

In [12]:
docs = [index[k] for k in index]

In [13]:
reverse_index = {}

for url in texts_lemmatized:
    for word in texts_lemmatized[url]:
        for word1 in word.split():
            if not word1 in reverse_index:
                 # если слово не входит в reverse_index, то добавляем его туда
                reverse_index[word1] = set([url])
            else:
                reverse_index[word1].update([url])

In [14]:
logic = {
    '&': lambda x,y: x & y,
    '|': lambda x,y: x | y,
    '!': lambda x: set([doc for doc in docs if doc not in x])
}

def find(query):
    query = query.replace('(', ' ( ').replace(')', ' ) ').replace('!','! ').split()
    
    indexes_words = [i for i,x in enumerate(query) if not x in ['!', '&', '|']]
    for i in indexes_words:
        query[i] = reverse_index[query[i]]
    
    indexes_not = [i for i,x in enumerate(query) if x == '!']
    for i in indexes_not:
        x = query[i+1]
        query[i+1] = logic['!'](x)
        query.pop(i)
    
    indexes_and = [i for i,x in enumerate(query) if x == '&']
    for i in indexes_and:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['&'](x,y)
        query.pop(i)
        query.pop(i)
    
    indexes_or = [i for i,x in enumerate(query) if x == '|']
    for i in indexes_or:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['|'](x,y)
        query.pop(i)
        query.pop(i)
    
    return query[0]

In [15]:
x1 = random.choice(list(reverse_index.keys()))
x2 = random.choice(list(reverse_index.keys()))
x3 = random.choice(list(reverse_index.keys()))

In [18]:
query = f'{x1} | {x2} & !{x3}'
print(query)

встречавшимися | немало & !резервируемым


In [19]:
find(query)

{'http://freelance.habr.com/tasks/357021?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob',
 'https://m.habr.com/ru/company/funcorp/blog/545350/',
 'https://m.habr.com/ru/company/otus/blog/549814/',
 'https://m.habr.com/ru/company/zyxel/blog/549850/',
 'https://m.habr.com/ru/post/549818/',
 'https://m.habr.com/ru/post/549820/comments/'}

# Задание №4

In [20]:
tfs = {}

for word in reverse_index:
    for doc in docs:
        tokens = texts[doc].split()
        tf = tokens.count(word) / len(tokens)
        if not word in tfs:
            tfs[word] = {}
        tfs[word][doc] = tf

In [21]:
with open('task4_tfs.txt', 'w') as f:
    for word in reverse_index:
        for doc in docs:
            f.write(f"{word} {doc} {tfs[word][doc]}\n")

In [22]:
pd.DataFrame(tfs).T

,http://freelance.habr.com/tasks/357019?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,http://freelance.habr.com/tasks/357021?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,http://freelance.habr.com/tasks/357023?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,http://freelance.habr.com/tasks/357031?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,http://freelance.habr.com/tasks/357033?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,https://account.habr.com/info/agreement,https://account.habr.com/info/confidential/,https://company.habr.com/,https://freelance.habr.com/tasks?utm_campaign=tasks_postlist&amp;utm_content=block_name&amp;utm_medium=habr_block&amp;utm_source=habr_mob,https://freelance.habr.com/tasks?utm_campaign=tasks_postlist&amp;utm_content=tasks_all&amp;utm_medium=habr_block&amp;utm_source=habr_mob,...,https://m.habr.com/ru/users/alderserg/,https://m.habr.com/ru/users/dyakonov_design/,https://m.habr.com/ru/users/fokus-lop/,https://telegram.me/habr_com,https://tmtm.ru/services/advertising/,https://tmtm.ru/services/corpblog/,https://tmtm.ru/workshops/,https://twitter.com/habr_com,https://www.youtube.com/channel/UCd_sTwKqVrweTt4oAKY5y4w,https://zen.yandex.ru/habr
Все,0.004049,0.003268,0.004739,0.003817,0.00339,0.000854,0.000000,0.000000,0.002703,0.002703,...,0.009901,0.009091,0.008403,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
всё,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.002695,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
публикации,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.004255,0.006135,0.0,0.000000,0.002392
публикация,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
публикацию,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
публикацией,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
публикациею,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
публикаций,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.006383,0.000000,0.0,0.000000,0.000000
публикациям,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
публикациями,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [23]:
idfs = {}

for word in reverse_index:
    idfs[word] = math.log(len(docs) / len(reverse_index[word]))

In [24]:
with open('task4_idfs.txt', 'w') as f:
    for word in reverse_index:
        f.write(f"{word} {idfs[word]}\n")

In [25]:
pd.DataFrame(idfs,index=['']).T

,
Все,0.105361
всё,0.051293
публикации,0.162519
публикация,0.162519
публикацию,0.162519
публикацией,0.162519
публикациею,0.162519
публикаций,0.162519
публикациям,0.162519
публикациями,0.162519


In [26]:
tfidfs = {}

for word in tfs:
    for doc in tfs[word]:
        tfidf = tfs[word][doc] * idfs[word]
        if not word in tfidfs:
            tfidfs[word] = {}
        tfidfs[word][doc] = tfidf

In [27]:
with open('task4_tfidfs.txt', 'w') as f:
    for word in reverse_index:
        for doc in docs:
            f.write(f"{word} {doc} {tfidfs[word][doc]}\n")

In [28]:
pd.DataFrame(tfidfs).T

,http://freelance.habr.com/tasks/357019?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,http://freelance.habr.com/tasks/357021?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,http://freelance.habr.com/tasks/357023?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,http://freelance.habr.com/tasks/357031?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,http://freelance.habr.com/tasks/357033?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,https://account.habr.com/info/agreement,https://account.habr.com/info/confidential/,https://company.habr.com/,https://freelance.habr.com/tasks?utm_campaign=tasks_postlist&amp;utm_content=block_name&amp;utm_medium=habr_block&amp;utm_source=habr_mob,https://freelance.habr.com/tasks?utm_campaign=tasks_postlist&amp;utm_content=tasks_all&amp;utm_medium=habr_block&amp;utm_source=habr_mob,...,https://m.habr.com/ru/users/alderserg/,https://m.habr.com/ru/users/dyakonov_design/,https://m.habr.com/ru/users/fokus-lop/,https://telegram.me/habr_com,https://tmtm.ru/services/advertising/,https://tmtm.ru/services/corpblog/,https://tmtm.ru/workshops/,https://twitter.com/habr_com,https://www.youtube.com/channel/UCd_sTwKqVrweTt4oAKY5y4w,https://zen.yandex.ru/habr
Все,0.000427,0.000344,0.000499,0.000402,0.000357,0.000090,0.000000,0.000000,0.000285,0.000285,...,0.001043,0.000958,0.000885,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
всё,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000138,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
публикации,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000692,0.000997,0.0,0.000000,0.000389
публикация,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
публикацию,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
публикацией,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
публикациею,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
публикаций,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001037,0.000000,0.0,0.000000,0.000000
публикациям,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
публикациями,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


# Задание №5

In [29]:
docs_length = {}
for row in pd.DataFrame(tfidfs).iterrows():
    sum_len = 0
    for word in row[1]:
        sum_len += word**2
    docs_length[row[0]] = math.sqrt(sum_len)

In [30]:
lemmatizer = Mystem()

In [37]:
query_word = "публикации"
query = get_sparsed_list(query_word.lower().split())
query = " ".join([line.rstrip() for line in query])
query = query + ' ' + query_word.title()

In [38]:
tfs_query = {}
for word in query.split():
    for doc in docs:
        tokens = texts[doc].split()
        tf = tokens.count(word) / len(tokens)
        if not word in tfs_query:
            tfs_query[word] = {}
        tfs_query[word][doc] = tf

In [39]:
idfs_query = {}

for word in query.split():
    idfs_query[word] = math.log(len(docs) / len(reverse_index[word]))

In [40]:
pd.DataFrame(idfs_query,index=['']).T

,
публикации,0.162519
публикация,0.162519
публикацию,0.162519
публикацией,0.162519
публикациею,0.162519
публикаций,0.162519
публикациям,0.162519
публикациями,0.162519
публикациях,0.162519
Публикации,0.198451


In [41]:
tfidfs_query = {}

for word in tfs_query:
    for doc in tfs_query[word]:
        tfidf = tfs_query[word][doc] * idfs_query[word]
        if not word in tfidfs_query:
            tfidfs_query[word] = {}
        tfidfs_query[word][doc] = tfidf

In [42]:
pd.DataFrame(tfidfs_query).T

,http://freelance.habr.com/tasks/357019?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,http://freelance.habr.com/tasks/357021?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,http://freelance.habr.com/tasks/357023?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,http://freelance.habr.com/tasks/357031?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,http://freelance.habr.com/tasks/357033?utm_campaign=tasks_postlist&amp;utm_content=course&amp;utm_medium=habr_block&amp;utm_source=habr_mob,https://account.habr.com/info/agreement,https://account.habr.com/info/confidential/,https://company.habr.com/,https://freelance.habr.com/tasks?utm_campaign=tasks_postlist&amp;utm_content=block_name&amp;utm_medium=habr_block&amp;utm_source=habr_mob,https://freelance.habr.com/tasks?utm_campaign=tasks_postlist&amp;utm_content=tasks_all&amp;utm_medium=habr_block&amp;utm_source=habr_mob,...,https://m.habr.com/ru/users/alderserg/,https://m.habr.com/ru/users/dyakonov_design/,https://m.habr.com/ru/users/fokus-lop/,https://telegram.me/habr_com,https://tmtm.ru/services/advertising/,https://tmtm.ru/services/corpblog/,https://tmtm.ru/workshops/,https://twitter.com/habr_com,https://www.youtube.com/channel/UCd_sTwKqVrweTt4oAKY5y4w,https://zen.yandex.ru/habr
публикации,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.0,0.0,0.000692,0.000997,0.0,0.0,0.000389
публикация,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
публикацию,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
публикацией,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
публикациею,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
публикаций,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.0,0.0,0.001037,0.000000,0.0,0.0,0.000000
публикациям,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
публикациями,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
публикациях,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
Публикации,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00393,0.003608,0.003335,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000


In [43]:
tfidfs_query_flat = pd.DataFrame(tfidfs_query).T.sum().to_dict()

In [44]:
sum_len = 1
for doc in tfidfs_query_flat:
    sum_len += tfidfs_query_flat[doc]**2
query_length = math.sqrt(sum_len)

In [45]:
cos_sim = {}

for doc in docs:
    my_list = list()
    for word in query.split(' '):
        my_list.append(((pd.Series(tfidfs_query_flat) * tfs[word][doc]).sum()) / (docs_length[doc] * query_length))
    cos_sim[doc]=max(my_list)

In [47]:
Counter(cos_sim).most_common(100)

[('https://m.habr.com/ru/users/alderserg/', 0.025810867065965196),
 ('https://m.habr.com/ru/users/LuxoftRussia/', 0.02185275361171674),
 ('https://m.habr.com/ru/flows/popsci/', 0.01931784803228524),
 ('https://m.habr.com/ru/flows/develop/', 0.018214080433418482),
 ('https://m.habr.com/ru/flows/management/', 0.017589665988977498),
 ('https://m.habr.com/ru/users/fokus-lop/', 0.01664527036654577),
 ('https://m.habr.com/ru/flows/admin/', 0.015964954314066516),
 ('https://m.habr.com/ru/users/Zyxel_CIS/', 0.015151867633205153),
 ('https://m.habr.com/ru/users/dyakonov_design/', 0.014885657415596325),
 ('https://m.habr.com/ru/users/Seleditor/', 0.013155251161609576),
 ('https://m.habr.com/ru/flows/design/', 0.012825027630426117),
 ('https://m.habr.com/ru/users/Picard/', 0.011216986923876146),
 ('https://m.habr.com/ru/users/MaxRokatansky/', 0.011120861048248267),
 ('https://m.habr.com/ru/users/Reformat/', 0.009682768125204732),
 ('https://m.habr.com/ru/post/549842/comments/', 0.0091169858477764